# Algoritmo LU para inversión de matrices con paralelismo

Importes necesarios

In [1]:
import numpy as np
import threading
import time

Definimos la función para leer la matriz de un archivo .csv

In [2]:
def leer_csv(nombre_archivo):
    try:
        A = np.loadtxt(nombre_archivo, delimiter=",")
        if A.shape[0] != A.shape[1]:
            raise ValueError("La matriz no es cuadrada")
        return A
    except Exception as e:
        print(f"Error al leer el archivo CSV: {e}")
        return None

Definimos la función que hace la descomposición LU de la matriz original

In [3]:
def lu_decomposition(A):
    N = A.shape[0]
    L = np.zeros((N, N))
    U = np.zeros((N, N))

    for i in range(N):
        for k in range(i, N):
            suma = sum(L[i][j] * U[j][k] for j in range(i))
            U[i][k] = A[i][k] - suma

        for k in range(i, N):
            if i == k:
                L[i][i] = 1.0
            else:
                suma = sum(L[k][j] * U[j][i] for j in range(i))
                L[k][i] = (A[k][i] - suma) / U[i][i]

    return L, U

Definimos la función de sustitución hacia adelante

In [5]:
def forward_substitution(L, b):
    N = L.shape[0]
    y = np.zeros(N)
    for i in range(N):
        y[i] = b[i] - np.dot(L[i, :i], y[:i])
    return y

Definimos la función de sustitución hacia atrás

In [6]:
def backward_substitution(U, y):
    N = U.shape[0]
    x = np.zeros(N)
    for i in reversed(range(N)):
        x[i] = (y[i] - np.dot(U[i, i+1:], x[i+1:])) / U[i][i]
    return x

Definimos la función que invierte una columna específica de la matriz original

In [7]:
def invertir_columna(i, L, U, A_inv, N):
    e = np.zeros(N)
    e[i] = 1.0
    y = forward_substitution(L, e)
    x = backward_substitution(U, y)
    A_inv[:, i] = x

Definimos la función que hará la inversión de toda la matriz usando hilos para cada columna.

In [8]:
def invertir_matriz(A):
    N = A.shape[0]
    L, U = lu_decomposition(A)
    A_inv = np.zeros_like(A)

    hilos = []
    for i in range(N):
        hilo = threading.Thread(target=invertir_columna, args=(i, L, U, A_inv, N))
        hilos.append(hilo)
        hilo.start()

    for hilo in hilos:
        hilo.join()

    return A_inv

Probamos el algoritmo

In [10]:
A = leer_csv("matriz.csv")
if A is not None:
    N = A.shape[0]

    start = time.time()
    A_inv = invertir_matriz(A)
    end = time.time()

    if N <= 10:
        print("Matriz inversa A^-1:")
        print(np.round(A_inv, 6))
    else:
        np.savetxt("inversa.csv", A_inv, delimiter=",", fmt="%.10f")
        print("Matriz inversa guardada en 'inversa.csv' (no se muestra por ser mayor de 10x10).")

    print(f"\nTiempo de ejecución (solo inversión): {end - start:.6f} segundos")

Matriz inversa guardada en 'inversa.csv' (no se muestra por ser mayor de 10x10).

Tiempo de ejecución (solo inversión): 38.462295 segundos
